In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, window

spark = SparkSession.builder.appName("Uber_App").getOrCreate()
# df = spark.read.options(header="True", inferSchema="True").csv('/FileStore/tables/dataset-2.csv')
# or
df = spark.read.csv('/Users/suraj/Desktop/Documents/development_repo/pyspark/files/dataset.csv', header="True", inferSchema="True")
df.show()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/26 12:07:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+---------+------------+--------+------+---------------+--------+--------------+
|     Date|Time (Local)|Eyeballs|Zeroes|Completed Trips|Requests|Unique Drivers|
+---------+------------+--------+------+---------------+--------+--------------+
|10-Sep-12|           7|       5|     0|              2|       2|             9|
|10-Sep-12|           8|       6|     0|              2|       2|            14|
|10-Sep-12|           9|       8|     3|              0|       0|            14|
|10-Sep-12|          10|       9|     2|              0|       1|            14|
|10-Sep-12|          11|      11|     1|              4|       4|            11|
|10-Sep-12|          12|      12|     0|              2|       2|            11|
|10-Sep-12|          13|       9|     1|              0|       0|             9|
|10-Sep-12|          14|      12|     1|              0|       0|             9|
|10-Sep-12|          15|      11|     2|              1|       2|             7|
|10-Sep-12|          16|    

In [2]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Time (Local): integer (nullable = true)
 |-- Eyeballs: integer (nullable = true)
 |-- Zeroes: integer (nullable = true)
 |-- Completed Trips: integer (nullable = true)
 |-- Requests: integer (nullable = true)
 |-- Unique Drivers: integer (nullable = true)



In [ ]:
'''
Uber Data:
1) Which date had the most completed trips during the two-week period?
2) What was the highest number of completed trips within a 24-hour period?
3) Which hour of the day had the most requests during the two-week period?
'''

In [3]:
# 1) Which date had the most completed trips during the two-week period?

completed_trips = df.groupBy("Date").sum('Completed Trips')
completed_trips.orderBy(col("sum(Completed Trips)").desc() ).select('Date').first()['Date']

# completed_trips.orderBy(col("sum(Completed Trips)").desc() ).show()

'22-Sep-12'

In [4]:
# 2) What was the highest number of completed trips within a 24-hour period?

from pyspark.sql.functions import count, sum, rank

# completed_trips_by_window = df.groupBy(window("Time (Local)", "24 hours"))
# completed_trips_by_window.agg(sum('Completed Trips')).show()
from pyspark.sql import Window
from pyspark.sql.types import IntegerType

trip_window = Window.partitionBy("Time (Local)").orderBy("Completed Trips")
df_with_rank = df.withColumn("sum", sum(col('Completed Trips')).over(trip_window))
# df_with_rank.show()

# completed_trips_by_window = df \
#     .groupBy("Time (Local)") \
#     .agg(sum("Completed Trips").alias("Total Completed Trips")) \
#     .orderBy("Total Completed Trips", ascending=False)

completed_trips_by_window = df \
    .groupBy(window("Time (Local)", "24 hours")) \
    .agg(sum("Completed Trips").alias("Total Completed Trips")) \
    .orderBy("Total Completed Trips", ascending=False)


# completed_trips_by_window.show()

# df.withColumn('time', rank().over(window)).show()
# df = df.withColumn("Completed Trips", df["Completed Trips"].cast(IntegerType()))
# df = df.filter(df["Completed Trips"].isNotNull())
# df.select('Completed Trips').show()

# Group the data by 24-hour windows and sum the completed trips
# completed_trips_by_window = df \
#     .groupBy(window("Time (Local)", "24 hours")) \
#     .agg(count("Completed Trips").alias("Total Completed Trips")) 
    # .orderBy("Total Completed Trips", ascending=False)
# display(completed_trips_by_window)



AnalysisException: [DATATYPE_MISMATCH.UNEXPECTED_INPUT_TYPE] Cannot resolve "window(Time (Local), 86400000000, 86400000000, 0)" due to data type mismatch: Parameter 1 requires the ("(TIMESTAMP OR TIMESTAMP WITHOUT TIME ZONE)" or "STRUCT<start: TIMESTAMP, end: TIMESTAMP>" or "STRUCT<start: TIMESTAMP_NTZ, end: TIMESTAMP_NTZ>") type, however "Time (Local)" has the type "INT".;
'Aggregate [window(Time (Local)#18, 86400000000, 86400000000, 0)], [window(Time (Local)#18, 86400000000, 86400000000, 0) AS window#96, sum(Completed Trips#21) AS Total Completed Trips#105L]
+- Relation [Date#17,Time (Local)#18,Eyeballs#19,Zeroes#20,Completed Trips#21,Requests#22,Unique Drivers#23] csv


In [9]:
# 3) Which hour of the day had the most requests during the two-week period?

most_requests = df.groupBy('Time (Local)').agg(sum("Requests").alias("total_request"))
most_requests_hr = most_requests.orderBy('total_request', ascending=False).first()['Time (Local)']
print('hour:',most_requests_hr)

# most_requests.show()


hour: 23


In [7]:
# 4) What percentages of all zeroes during the two-week period occurred on weekend (Friday at 5 pm to Sunday at 3 am)? 

from pyspark.sql.functions import dayofweek, hour, to_date

df = df.withColumn("converted_date", to_date("Date", "dd-MMM-yy"))   # date is in string format so changed
df = df.withColumn("week", dayofweek("converted_date"))

weekend_zeros = df.filter( (col("Time (Local)") >= 17) | (col("Time (Local)") < 3) ).filter((col("week") == 6) | (col("week") == 7) | (col("week") == 5))

weekend_zeros_count = weekend_zeros.agg(sum("Zeroes").alias("weekend_zeros")) \
                    .collect()[0]["weekend_zeros"]

print(weekend_zeros_count)
# weekend_zeros.show()
# df.printSchema()

total_zeros_count = df.agg(sum('Zeroes').alias('total_zeros') ).collect()[0]['total_zeros']
print(total_zeros_count)

percentages_zeros = (weekend_zeros_count/total_zeros_count)*100
print(percentages_zeros)


496
1429
34.70958712386284


In [ ]:
'''
5) What is the weighted average ratio of completed trips per driver during the two-week period? 
Tip: “Weighted average” means your answer should account for the total trip volume in each hour to determine the most accurate number in the whole period.

6) In drafting a driver schedule in terms of 8 hours shifts, when are the busiest 8 consecutive hours over the two-week period in terms of unique requests? 
A new shift starts every 8 hours. Assume that a driver will work the same shift each day.

7) True or False: Driver supply always increases when demand increases during the two-week period. Tip: Visualize the data to confirm your answer if needed.
'''

In [20]:
# 5) What is the weighted average ratio of completed trips per driver during the two-week period? 
# Tip: “Weighted average” means your answer should account for the total trip volume in each hour to determine the most accurate number in the whole period.

from pyspark.sql.functions import avg

# a= (completed trip / unique driver, total no of completed trips for that hour, then sum result) // total no. of completed trips

weighted_avg = df.withColumn("completed_per_driver", df["Completed Trips"] / df["Unique Drivers"]) \
                 .groupBy("Date", "Time (Local)").agg(avg("completed_per_driver").alias("avg_completed_per_driver"), sum("Completed Trips").alias("total_completed_trips") )
# weighted_avg.show()

weighted_ratio = weighted_avg.withColumn("weighted_ratio", col("avg_completed_per_driver") * col("total_completed_trips") )
# weighted_ratio.show()

weighted_avg_ratio = weighted_ratio.agg(sum("weighted_ratio") / sum("total_completed_trips"))  # .collect()[0][0]
weighted_avg_ratio.show()
print(weighted_avg_ratio.collect()[0][0])


+--------------------------------------------------+
|(sum(weighted_ratio) / sum(total_completed_trips))|
+--------------------------------------------------+
|                                0.8276707747535554|
+--------------------------------------------------+

0.8276707747535554


In [28]:
# 6) In drafting a driver schedule in terms of 8 hours shifts, when are the busiest 8 consecutive hours over the two-week period in terms 
# of unique requests? 
# A new shift starts every 8 hours. Assume that a driver will work the same shift each day.

from pyspark.sql.functions import col, hour, countDistinct
from pyspark.sql.window import Window

hourly_unique_requests = df.groupby(col("Time (Local)").alias('hour')).agg(countDistinct('Requests').alias('unique_requests'))
# hourly_unique_requests.show()

# Slide a window of 8 hours to find the busiest 8 consecutive hours
window = Window.orderBy(col("unique_requests").desc()).rowsBetween(0, 7)

busiest_8_consecutive_hours = (hourly_unique_requests
  .select("*", sum("unique_requests").over(window).alias("consecutive_sum"))
  .orderBy(col("consecutive_sum").desc())
  .limit(1)
)

busiest_8_consecutive_hours.show()

24/12/26 16:30:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/26 16:30:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/26 16:30:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/26 16:30:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/26 16:30:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----+---------------+---------------+
|hour|unique_requests|consecutive_sum|
+----+---------------+---------------+
|  20|             12|             80|
+----+---------------+---------------+



24/12/26 16:30:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/26 16:30:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/26 16:30:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/26 16:30:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [ ]:
# 7) True or False: Driver supply always increases when demand increases during the two-week period. Tip: Visualize the data to confirm 
# your answer if needed.
False


In [11]:
from datetime import datetime
a= '10-Sep-12'
# b = datetime.date(a)

print(dayofweek('2012-09-10'))

Column<'dayofweek(2012-09-10)'>


In [ ]:
'''
8) In which 72-hour period is the ratio of Zeroes to Eyeballs the highest?

9) If you could add 5 drivers to any single hour of every day during the two-week period, which hour should you add them to? 
Hint: Consider both rider eyeballs and driver supply when choosing.

10) Looking at the data from all two weeks, which time might make the most sense to consider a true “end day” instead of midnight? 
(i.e when are supply and demand at both their natural minimums)
'''

In [40]:
# 8) In which 72-hour period is the ratio of Zeroes to Eyeballs the highest?

# Group the data by 72-hour periods and calculate the ratio of zeroes to eyeballs for each period
period_ratios = (df
  .groupBy(((col("Date").cast("timestamp").cast("long") / (72*3600)).cast("int")).alias("period"))
  .agg(sum("Zeroes").alias("zeroes"), sum("Eyeballs").alias("eyeballs"))
  .withColumn("ratio", col("zeroes") / col("eyeballs"))
)
period_ratios.show()

# Find the period with the highest ratio
highest_ratio_period = period_ratios.orderBy(col("ratio").desc()).limit(1)
print(highest_ratio_period)

# trip_window = Window.orderBy("Date").rowsBetween(0,3)
# window = Window.orderBy(col("unique_requests").desc()).rowsBetween(0, 7)
# period_ratios = df.select("*", sum("Zeroes").over(trip_window), sum("Eyeballs").over(trip_window) )

# period_ratios = df.groupby(trip_window).agg(sum("Zeroes"))
# period_ratios.show()



+------+------+--------+-------------------+
|period|zeroes|eyeballs|              ratio|
+------+------+--------+-------------------+
|  NULL|  1429|    6687|0.21369822042769554|
+------+------+--------+-------------------+

DataFrame[period: int, zeroes: bigint, eyeballs: bigint, ratio: double]


In [45]:
# 9) If you could add 5 drivers to any single hour of every day during the two-week period, which hour should you add them to? 
# Hint: Consider both rider eyeballs and driver supply when choosing.

ratio = df.groupby(col("Time (Local)").alias("hour") ).agg((sum("Requests")/sum("Unique Drivers")).alias("request_driver_ratio") )
# ratio.show()

ratio.orderBy(col("request_driver_ratio").desc() ).collect()[0]


Row(hour=2, request_driver_ratio=1.6129032258064515)

In [56]:
# 10) Looking at the data from all two weeks, which time might make the most sense to consider a true “end day” instead of midnight? 
# (i.e when are supply and demand at both their natural minimums)
from pyspark.sql.functions import col, mean

avg_trips_and_drivers = df.groupby(col("Time (Local)").alias("hour") ).agg(avg("Completed Trips").alias("avg_completed_trips"), avg("Unique Drivers").alias("avg_unique_drivers"))
avg_trips_and_drivers.show()

avg_trips_and_drivers.orderBy('avg_completed_trips', 'avg_unique_drivers').show(1)


+----+-------------------+------------------+
|hour|avg_completed_trips|avg_unique_drivers|
+----+-------------------+------------------+
|  12| 3.2857142857142856| 9.428571428571429|
|  22|  9.571428571428571|10.285714285714286|
|   1|  5.071428571428571| 6.714285714285714|
|  13|  3.142857142857143| 8.714285714285714|
|  16|  4.857142857142857|10.285714285714286|
|   6| 1.3571428571428572| 2.642857142857143|
|   3|                1.5| 2.857142857142857|
|  20|  5.428571428571429|11.642857142857142|
|   5| 0.2857142857142857|0.7857142857142857|
|  19|  8.571428571428571|12.857142857142858|
|  15|  3.357142857142857| 9.928571428571429|
|   9| 1.4285714285714286| 7.857142857142857|
|  17|  5.571428571428571|11.785714285714286|
|   4|0.14285714285714285|0.6428571428571429|
|   8| 1.7142857142857142| 6.785714285714286|
|  23|                9.0|               8.5|
|   7| 1.3571428571428572| 4.285714285714286|
|  10| 1.2857142857142858| 9.214285714285714|
|  21|  6.285714285714286|11.07142